Highlighter 

In [23]:
from transformers import AutoTokenizer, AutoModel, AutoConfig
import torch
import numpy as np

model_name = 'gpt2'
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
text = "e.g. 0000 -> 1111 + 0001 -> 0000"
text_split = text.split(" ")
print(text_split)
tokenizer.pad_token = tokenizer.eos_token
encoded_input = tokenizer(text, return_tensors='pt')['input_ids']
print(encoded_input.shape)
tokens = tokenizer.batch_decode(torch.squeeze(encoded_input))
print(tokens)
output = model(encoded_input)
print(output)
output_vecs = torch.squeeze(output.last_hidden_state)
output_vecs = output_vecs.detach().numpy()
print(output_vecs)

i = 0
while (i < len(tokens) - 1):
    if (tokens[i].strip() != text_split[i]):
        tokens[i] = tokens[i] + tokens[i + 1]
        output_vecs[i] = output_vecs[i] + output_vecs[i + 1]
        tokens.pop(i + 1)
        output_vecs = np.delete(output_vecs, [i + 1], axis=0)
    else:
        i += 1

output_vecs = torch.from_numpy(output_vecs)
print(output_vecs.shape)
print(tokens)

['e.g.', '0000', '->', '1111', '+', '0001', '->', '0000']
torch.Size([1, 13])
['e', '.', 'g', '.', ' 0000', ' ->', ' 111', '1', ' +', ' 00', '01', ' ->', ' 0000']
BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-0.0789, -0.1071, -0.3308,  ..., -0.1371, -0.0112,  0.0291],
         [ 0.1726, -0.9235,  0.3001,  ...,  0.1390,  0.2802,  0.0465],
         [ 0.5592, -0.2487, -0.1439,  ..., -0.2402,  0.0605,  0.5375],
         ...,
         [ 0.3621, -0.0544, -0.3518,  ...,  0.4484,  0.0706, -0.3354],
         [ 0.6125, -0.2568, -0.3509,  ...,  0.0240,  0.5127, -0.4606],
         [ 0.7557, -0.3836, -0.2043,  ...,  0.2898,  0.3580, -0.4222]]],
       grad_fn=<ViewBackward0>), past_key_values=((tensor([[[[-1.1873,  2.5328,  0.8507,  ..., -1.4194, -0.3196,  2.0127],
          [-2.1205,  2.9310,  1.7164,  ..., -0.2185, -1.5958,  1.6449],
          [-2.7917,  2.6574,  2.1891,  ..., -1.3604, -1.6030,  2.8977],
          ...,
          [-2.0684,  3.4399,  2.6216,  ..., -1.3287, 

In [3]:
from highlight import HighlightHead

hh = HighlightHead(outdim=4,config=config)
print(hh(output_vecs))

tensor([[0.0571, 0.1366, 0.1117, 0.0982],
        [0.1042, 0.1090, 0.0902, 0.0891],
        [0.1242, 0.0993, 0.0898, 0.1027],
        [0.1205, 0.1045, 0.1165, 0.1161],
        [0.1038, 0.1014, 0.1084, 0.1101],
        [0.1172, 0.0873, 0.1318, 0.1212],
        [0.1218, 0.1109, 0.1215, 0.1234],
        [0.1507, 0.1430, 0.1187, 0.1290],
        [0.1004, 0.1081, 0.1114, 0.1102]], grad_fn=<SoftmaxBackward0>)


In [7]:
import pandas as pd

df = pd.read_csv("data.csv", header=None)
display(df)

                                                    0                     1   \
0                                    Binary Arithmetic  [1.0, 0.0, 0.0, 0.0]   
1                                      Signed Integers  [1.0, 0.0, 0.0, 0.0]   
2    Can’t simply add lead bit indicating sign beca...  [1.0, 0.0, 0.0, 0.0]   
3    Two’s Complement - Start with the positive num...  [0.0, 0.0, 1.0, 0.0]   
4                                 Throw away carry out  [1.0, 0.0, 0.0, 0.0]   
..                                                 ...                   ...   
612                                   Search Algorithm  [0.0, 0.0, 1.0, 0.0]   
613          Iterate through string and compare hashes  [0.0, 0.0, 0.0, 1.0]   
614                  If same hash, then compare string  [0.0, 0.0, 0.0, 1.0]   
615                      Else, update hash and iterate  [0.0, 0.0, 0.0, 1.0]   
616                Big O is because of hash collisions  [1.0, 0.0, 0.0, 0.0]   

                       2               